In [1]:
import pandas as pd
import re
import os
import glob
import concurrent.futures
from tqdm import tqdm
import time
import numpy as np
import concurrent.futures

# COMPANY DATA METHOD NEW

In [48]:
# create column names and dataframe

path = 'sec_edgar_filings/917520/10-K/0000917520-20-000010.txt'
file = open(path)
lines = [line for line in file.readlines() if line.strip()]
lines = [x.strip() for x in lines]
for i in range(len(lines)):
        try:
            lines[i] = re.sub('\t+','',lines[i])
        except:
            pass
file.close()

header_row = lines.index('</SEC-HEADER>')
companystart = lines.index('FILER:') + 2
lines = lines[companystart:header_row]
col = []
for i in lines:
    col.append(i.split(':')[0])
col.insert(13,'STREET 2')
col.insert(20,'STREET 2')
for ii in range(9):
    col.append(col[24] + ' ' + f'{ii + 2}')
    col.append(col[25] + ' ' + f'{ii + 2}')
    col.append(col[26] + ' ' + f'{ii + 2}')

df = pd.DataFrame(index = col)

In [49]:
# create function that find the first/second occurence of a specified char in string

def find_2nd(string, substring):
    return string.find(substring, string.find(substring) + 1)
def find_1st(string, substring):
    return string.find(substring, string.find(substring))

In [50]:
# integrate paths and modify it to be linux/os/windows compatible

paths10k = glob.glob('sec_edgar_filings/*/10-K/*.txt')
paths8k = glob.glob('sec_edgar_filings/*/8-K/*.txt')
pathsSC = glob.glob('sec_edgar_filings/*/SC 13G/*.txt')

paths = paths10k + paths8k + pathsSC
for i in range(len(paths)):
    try:
        paths[i] = paths[i].replace('\\','/')
    except:
        pass

In [51]:
# only select one path from each company, can be from SG 13C or 8-K or 10-K

path = []
for i in range(len(paths)):
    det = True
    for ii in path:
        if paths[i][find_1st(paths[i],'/')+1 : find_2nd(paths[i],'/')] in ii:
            det = False
    if det:
        path.append(paths[i])

In [52]:
# integrate all to be one big df

not_in_format = []
for p in tqdm(path):
    try:
        file = open(p)
        lines = [line for line in file.readlines() if line.strip()]
        lines = [x.strip() for x in lines]
        for i in range(len(lines)):
                try:
                    lines[i] = re.sub('\t+','',lines[i])
                except:
                    pass
        file.close()

        try:
            FB_row = lines.index('FILED BY:')
        except:
            FB_row = 9999
        header_row = lines.index('</SEC-HEADER>')
        if FB_row > header_row:
            end = header_row
        else:
            end = FB_row
        companystart = lines.index('COMPANY DATA:') + 1
        lines = lines[companystart:end]
        for i in range(len(col[:22])):
            if lines[i].split(':')[0] != col[i]:
                lines.insert(i,col[i] + ':')
        
        
        values = []
        for i in lines:
            values.append(i.split(':')[1])
        if len(values) != df.shape[0]:
            for i in range(df.shape[0]-len(values)):
                values.append('None')
        if lines.index('MAIL ADDRESS:') != 18:
            not_in_format.append(values[1])
            continue
        df[f'{values[1]}'] = values
    except:
        continue


  3%|▎         | 313/11758 [04:01<3:39:15,  1.15s/it]


  5%|▌         | 623/11758 [08:17<3:54:29,  1.26s/it]


  8%|▊         | 930/11758 [12:17<2:25:20,  1.24it/s]


 11%|█         | 1246/11758 [15:57<1:43:03,  1.70it/s]


 13%|█▎        | 1558/11758 [19:33<2:53:47,  1.02s/it]


...

 48%|████▊     | 5675/11758 [1:05:15<1:46:49,  1.05s/it]


 51%|█████     | 5962/11758 [1:09:08<1:18:04,  1.24it/s]


 56%|█████▋    | 6628/11758 [1:12:49<12:58,  6.59it/s]


 84%|████████▎ | 9821/11758 [1:14:41<00:15, 126.35it/s]


100%|██████████| 11758/11758 [1:14:54<00:00,  2.62it/s] 

In [55]:
df.T.to_csv('companydata.csv')


In [53]:
len(not_in_format)

0

In [56]:
df

,0001000045,0001000209,0001000228,0001000229,0001000230,0001000232,0001000298,0001000623,0001000683,0001000694,...,0001072019,0001532543,0001020897,0000942149,0000887546,0001031516,0000723733,0000946770,0000832812,0001089044
COMPANY CONFORMED NAME,NICHOLAS FINANCIAL INC,MEDALLION FINANCIAL CORP,HENRY SCHEIN INC,CORE LABORATORIES N V,OPTICAL CABLE CORP,KENTUCKY BANCSHARES INC /KY/,IMPAC MORTGAGE HOLDINGS INC,SCHWEITZER MAUDUIT INTERNATIONAL INC,BLONDER TONGUE LABORATORIES INC,NOVAVAX INC,...,American Bonanza Gold Corp.,ROI Acquisition Corp.,SPECIALTY CATALOG CORP,MIRANDA GOLD CORP,GREATER CHINA FUND INC,Merus Labs International Inc.,"MULTI SOLUTIONS II, INC",DEUTSCHE TELEKOM AG,KINDERCARE LEARNING CENTERS INC /DE,LABRANCHE & CO INC
CENTRAL INDEX KEY,0001000045,0001000209,0001000228,0001000229,0001000230,0001000232,0001000298,0001000623,0001000683,0001000694,...,0001072019,0001532543,0001020897,0000942149,0000887546,0001031516,0000723733,0000946770,0000832812,0001089044
STANDARD INDUSTRIAL CLASSIFICATION,SHORT-TERM BUSINESS CREDIT INSTITUTIONS [6153],FINANCE SERVICES [6199],"WHOLESALE-MEDICAL, DENTAL & HOSPITAL EQUIPMENT...","OIL, GAS FIELD SERVICES, NBC [1389]",DRAWING AND INSULATING NONFERROUS WIRE [3357],STATE COMMERCIAL BANKS [6022],REAL ESTATE INVESTMENT TRUSTS [6798],PAPER MILLS [2621],RADIO & TV BROADCASTING & COMMUNICATIONS EQUIP...,BIOLOGICAL PRODUCTS (NO DIAGNOSTIC SUBSTANCES)...,...,METAL MINING [1000],BLANK CHECKS [6770],RETAIL-CATALOG & MAIL-ORDER HOUSES [5961],METAL MINING [1000],UNKNOWN SIC - 0000 [0000],PHARMACEUTICAL PREPARATIONS [2834],BLANK CHECKS [6770],RADIO TELEPHONE COMMUNICATIONS [4812],SERVICES-CHILD DAY CARE SERVICES [8351],"SECURITY BROKERS, DEALERS & FLOTATION COMPANIE..."
IRS NUMBER,593019317,043291176,113136595,000000000,541237042,610993464,330675505,621612879,521611421,222816046,...,000000000,453414553,043253301,000000000,133672942,000000000,222418056,000000000,630941966,134064735
STATE OF INCORPORATION,FL,DE,DE,P7,VA,KY,MD,DE,DE,DE,...,A1,DE,DE,A1,MD,A1,FL,,DE,DE
FISCAL YEAR END,0331,1231,1228,1231,1031,1231,1231,1231,1231,1231,...,1231,0331,1231,0831,1231,0930,0131,,0531,1231
FILING VALUES,,,,,,,,,,,...,,,,,,,,,,
FORM TYPE,SC 13G,10-K,SC 13G,10-K,10-K,10-K,10-K,SC 13G,SC 13G,10-K,...,SC 13G,SC 13G,SC 13G,SC 13G,SC 13G,SC 13G,SC 13G,SC 13G,SC 13G,SC 13G
SEC ACT,1934 Act,1934 Act,1934 Act,1934 Act,1934 Act,1934 Act,1934 Act,,1934 Act,1934 Act,...,1934 Act,1934 Act,,1934 Act,1934 Act,1934 Act,1934 Act,1934 Act,1934 Act,1934 Act
SEC FILE NUMBER,005-59007,001-37747,005-46221,001-14273,000-27022,000-52598,001-14100,005-48407,005-52487,000-26770,...,005-80357,005-86748,005-50224,005-83177,005-51363,005-81986,005-35524,005-84024,005-40318,005-56767


# File data

In [2]:
def get_file(path):
    infofile = []
    infoFB = []
#     tmp = []
    for i in path:
        x = i.split('/')[1]
        
        if 'SC 13G' not in i:
            with open(i, 'r') as f:
                info = f.readlines()
            info = [x.rstrip('\n') for x in info]
            info = [x.strip('\t') for x in info]
            fileend = info.index('FILER:')
            tmp = info[:fileend]
            tmp = ['Filing_Company_CIK:' + x] + tmp
        else:
            with open(i, 'r') as f:
                infoSC = f.readlines()
            infoSC = [x.rstrip('\n') for x in infoSC]
            infoSC = [x.strip('\t') for x in infoSC]
            
            fileend = infoSC.index('SUBJECT COMPANY:') - 1
            tmp = infoSC[:fileend]
            tmp = ['Filing_Company_CIK:' + x] + tmp
            
            FBstart = infoSC.index('FILED BY:')
            header_row = infoSC.index('</SEC-HEADER>')
            infoFB = infoSC[FBstart:header_row]
#             infoFB = ['Primary_CIK:' + x] + infoFB
            
        infofile.append(tmp)
    
    return (infofile, infoFB)

In [3]:
def cleaning_trans_comp(info):

# remove NAs and tabs
    info = [x for x in info if x == x]

    for i in range(len(info)):
        try:
            info[i] = re.sub('\t+','',info[i])
        except:
            pass
    
# distinguish duplicated keys in different files
    for i in range(len(info)):
        for ii in range(i+1,len(info)):
            if (info[i].split(':')[0] + ':') in info[ii] and info[i] != '':
                info[ii] = info[ii].split(':')[0] + '_' + str( "%04d" % ii) + ':' + info[ii].split(':')[1]

# transform to dictionary
    dic = {}
    for i in info:
        try:
            dic.update({i.split(':')[0]:i.split(':')[1].strip('\t')})
        except:
            pass
        
# transform to the dataframe
    df = pd.DataFrame(dic, index = [0])
    
# put the address together(street, city, state, zip)
    for i, col in enumerate(df.columns):
        if 'ADDRESS' in col:
            df[col] = df[col].str.cat(df.iloc[:,i+1:i+6],sep = ',')
            df.iloc[:, i+1:i+6] = ''

    return df

In [4]:
def cleaning_file(info):
# remove NAs and tabs
    info = [x for x in info if x == x]

    for i in range(len(info)):
        try:
            info[i] = info[i].replace('\t','')
        except:
            pass
    
    for i in range(len(info)):
        info[i] = info[i].lstrip()
        if '>' in info[i] and ':' not in info[i]:
            info[i] = info[i].replace('>', '>:')
        elif '>' in info[i] and ':' in info[i]:
            info[i] = info[i].replace(':',';')
            info[i] = info[i].replace('>', '>:')
              
    
# distinguish duplicated keys in different files
    for i in range(len(info)):
        for ii in range(i+1,len(info)):
            if (info[i].split(':')[0] + ':') in info[ii] and info[i] != '':
                info[ii] = info[ii].split(':')[0] + '_' + str( "%04d" % ii) + ':' + info[ii].split(':')[1]
                
                
    info = [x for x in info if x != '']

    return info

In [5]:
def trans(info):
# transform to dataframe
    df_file = pd.DataFrame()
    
    dic_file = {}
    idx = 0
    for k in info:
        for j in k:
            try:
                if j.split(':')[1] != '':
                    dic_file.update({j.split(':')[0]:j.split(':')[1]})
            except:
                pass
        tmp = pd.DataFrame(dic_file, index = [idx])

        df_file = df_file.join(tmp.T,how = 'outer')

        dic_file = {}
        idx += 1
    
    return df_file

In [88]:
companies = glob.glob('sec_edgar_filings/*')
len(companies)

17205

In [6]:
companies = glob.glob('sec_edgar_filings/*/*/*.txt')
companies[14369:14371]

['sec_edgar_filings/1433994/10-K/0001564590-19-007047.txt',
 'sec_edgar_filings/1433994/10-Q/0001564590-19-042369.txt']

In [6]:
datafile = pd.DataFrame()
companies = glob.glob('sec_edgar_filings/*')
jump = 0

# def final_function(companies):
for company in tqdm(companies):
    try:
        path = glob.glob(company + '/*/*.txt')

        infofile, infoFB = get_file(path)

        infofile = list(map(cleaning_file, infofile))
        infofile = trans(infofile)


        if infoFB!= []:
            infoFB = cleaning_trans_comp(infoFB)

            new = []
            for i in infoFB.columns:
                new.append('FILED BY:' + i)
            infoFB.columns = new

            infoFB.index = infofile.T[infofile.T['CONFORMED SUBMISSION TYPE'] == 'SC 13G'].index
            infofile = infofile.T.join(infoFB, how = 'left', lsuffix = 'CONFORMED SUBMISSION TYPE', rsuffix = 'FORM TYPE')

            result = infofile.T.drop_duplicates()
            df = result.T
        else:
            df = infofile.T.drop_duplicates()

        df.index = df.index.map(lambda x: x + jump)
        jump += len(df.T.columns)
        datafile = datafile.join(df.T, how = 'outer')
    except:
        pass
#     return datafile

# with concurrent.futures.ThreadPoolExecutor() as executor:
#     executor.map(final_function,companies)
datafile.to_csv('temp_filedata.csv', index = False)

100%|██████████| 17205/17205 [11:46:55<00:00,  5.01s/it]  


In [8]:
# reorder the columns    
cols = list(datafile.T.columns)

news = []
olds = []
for i in cols:
    if 'FILED BY:' in i:
        news.append(i)
    else:
        olds.append(i)
cols = olds + news
cols.insert(0,cols.pop(cols.index('CONFORMED SUBMISSION TYPE')))
cols.insert(0,cols.pop(cols.index('Filing_Company_CIK')))
cols.insert(cols.index('FILED BY:BUSINESS ADDRESS') ,cols.pop(cols.index('FILED BY:COMPANY CONFORMED NAME')))


final_file = datafile.T.loc[:,cols].drop(['FILED BY:FILED BY'], axis = 1)

In [9]:
final_file.to_csv('filedata.csv', index = False)

In [11]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
final_file.head()

,Filing_Company_CIK,CONFORMED SUBMISSION TYPE,<ABS-ASSET-CLASS>,<ABS-ASSET-CLASS>_0015,<ABS-SUB-ASSET-CLASS>,<ACCEPTANCE-DATETIME>,<DEPOSITOR-CIK>,<IMS-DOCUMENT>,<IMS-HEADER>,<ITEM-NUMBER>,<ITEM-NUMBER>_0014,<ITEM-PERIOD>,<ITEM-PERIOD>_0015,<REFERENCE-462B>,<RULE-NAME>,<SEC-DOCUMENT>,<SEC-HEADER>,<SPONSOR-CIK>,ABS ASSET CLASS,ABS ASSET CLASS_0013,ABS ASSET CLASS_0015,ACCESSION NUMBER,BUSINESS PHONE,CENTRAL INDEX KEY,CITY,CITY_0037,COMPANY CONFORMED NAME,CONFORMED PERIOD OF REPORT,DATE AS OF CHANGE,DATE OF NAME CHANGE,DATE OF NAME CHANGE_0043,DATE OF NAME CHANGE_0052,DATE OF NAME CHANGE_0053,EFFECTIVENESS DATE,FILED AS OF DATE,FISCAL YEAR END,FORM TYPE,FORMER CONFORMED NAME,FORMER CONFORMED NAME_0042,FORMER CONFORMED NAME_0051,FORMER CONFORMED NAME_0052,GROUP MEMBERS,GROUP MEMBERS_0010,GROUP MEMBERS_0011,GROUP MEMBERS_0012,GROUP MEMBERS_0013,GROUP MEMBERS_0014,GROUP MEMBERS_0015,GROUP MEMBERS_0016,GROUP MEMBERS_0017,GROUP MEMBERS_0018,GROUP MEMBERS_0019,GROUP MEMBERS_0020,GROUP MEMBERS_0021,GROUP MEMBERS_0022,GROUP MEMBERS_0023,GROUP MEMBERS_0024,GROUP MEMBERS_0025,GROUP MEMBERS_0026,GROUP MEMBERS_0027,GROUP MEMBERS_0028,GROUP MEMBERS_0029,GROUP MEMBERS_0030,GROUP MEMBERS_0031,GROUP MEMBERS_0032,GROUP MEMBERS_0033,GROUP MEMBERS_0034,GROUP MEMBERS_0035,GROUP MEMBERS_0036,GROUP MEMBERS_0037,GROUP MEMBERS_0038,GROUP MEMBERS_0039,GROUP MEMBERS_0040,GROUP MEMBERS_0041,GROUP MEMBERS_0042,GROUP MEMBERS_0043,GROUP MEMBERS_0044,GROUP MEMBERS_0045,GROUP MEMBERS_0046,GROUP MEMBERS_0047,GROUP MEMBERS_0048,GROUP MEMBERS_0049,GROUP MEMBERS_0050,GROUP MEMBERS_0051,GROUP MEMBERS_0052,GROUP MEMBERS_0053,GROUP MEMBERS_0054,GROUP MEMBERS_0055,IRS NUMBER,ITEM INFORMATION,ITEM INFORMATION_0009,ITEM INFORMATION_0010,ITEM INFORMATION_0011,ITEM INFORMATION_0012,ITEM INFORMATION_0013,ITEM INFORMATION_0014,ITEM INFORMATION_0015,ITEM INFORMATION_0016,ITEM INFORMATION_0018,ITEM INFORMATION_0019,ITEM INFORMATION_0020,ITEM INFORMATION_0021,ITEM INFORMATION_0022,ITEM INFORMATION_0023,ITEM INFORMATION_0024,ITEM INFORMATION_0025,ITEM INFORMATION_0026,MIC-Info,Originator-Name,PUBLIC DOCUMENT COUNT,Proc-Type,REFERENCES 429,REFERENCES 429_0008,REFERENCES 429_0009,REFERENCES 429_0010,REFERENCES 429_0011,REFERENCES 429_0012,REFERENCES 429_0013,REFERENCES 429_0014,REFERENCES 429_0015,REFERENCES 429_0016,REFERENCES 429_0017,REFERENCES 429_0018,REFERENCES 429_0019,REFERENCES 429_0020,REFERENCES 429_0021,REFERENCES 429_0022,REFERENCES 429_0023,REFERENCES 429_0024,REFERENCES 429_0025,REFERENCES 429_0026,SROS,SROS_0018,SROS_0019,SROS_0020,STANDARD INDUSTRIAL CLASSIFICATION,STATE,STATE OF INCORPORATION,STATE_0038,STREET 1,STREET 1_0028,STREET 1_0030,STREET 1_0031,STREET 1_0035,STREET 1_0036,STREET 1_0037,STREET 1_0039,STREET 1_0040,STREET 1_0041,STREET 1_0042,STREET 1_0045,STREET 2,STREET 2_0036,STREET 2_0037,STREET 2_0042,ZIP,ZIP_0039,ZIP_0045,FILED BY:COMPANY CONFORMED NAME,FILED BY:BUSINESS ADDRESS,FILED BY:BUSINESS ADDRESS_0030,FILED BY:BUSINESS ADDRESS_0031,FILED BY:BUSINESS ADDRESS_0032,FILED BY:BUSINESS ADDRESS_0033,FILED BY:BUSINESS ADDRESS_0034,FILED BY:BUSINESS ADDRESS_0035,FILED BY:BUSINESS ADDRESS_0036,FILED BY:BUSINESS ADDRESS_0037,FILED BY:BUSINESS ADDRESS_0038,FILED BY:BUSINESS ADDRESS_0039,FILED BY:BUSINESS ADDRESS_0040,FILED BY:BUSINESS ADDRESS_0041,FILED BY:BUSINESS ADDRESS_0043,FILED BY:BUSINESS ADDRESS_0044,FILED BY:BUSINESS ADDRESS_0046,FILED BY:BUSINESS ADDRESS_0048,FILED BY:BUSINESS ADDRESS_0050,FILED BY:BUSINESS ADDRESS_0051,FILED BY:BUSINESS PHONE,FILED BY:BUSINESS PHONE_0038,FILED BY:BUSINESS PHONE_0039,FILED BY:BUSINESS PHONE_0040,FILED BY:BUSINESS PHONE_0041,FILED BY:BUSINESS PHONE_0042,FILED BY:BUSINESS PHONE_0043,FILED BY:BUSINESS PHONE_0044,FILED BY:BUSINESS PHONE_0045,FILED BY:BUSINESS PHONE_0049,FILED BY:BUSINESS PHONE_0052,FILED BY:BUSINESS PHONE_0056,FILED BY:BUSINESS PHONE_0057,FILED BY:CENTRAL INDEX KEY,FILED BY:CENTRAL INDEX KEY_0013,FILED BY:CENTRAL INDEX KEY_0016,FILED BY:CENTRAL INDEX KEY_0017,FILED BY:CENTRAL INDEX KEY_0019,FILED 

## Please do not close the file

# END --------------------------------------------------------

## COMPANY DATA METHOD ORIGINAL
    --reading file method went wrong

In [35]:
def get_comp(path):
    infocomp = []
    for i in path:
        if 'SC 13G' not in i:
            df = pd.read_fwf(i,header = None)
            info = df[0].to_list()
            header_row = info.index('</SEC-HEADER>')
            companystart = info.index('FILER:') + 3
            infocomp += info[companystart:header_row]
        else:
            df = pd.read_fwf(i,header = None)
            infoSC = df[0].to_list() 
            companystart = infoSC.index('SUBJECT COMPANY:') + 3
            companyend = infoSC.index('FILED BY:') - 1
            infocomp += infoSC[companystart:companyend]
            
    return infocomp
    

In [36]:
def cleaning_comp(info):

# remove NAs and tabs
    info = [x for x in info if x == x]

    for i in range(len(info)):
        try:
            info[i] = re.sub('\t+','',info[i])
        except:
            pass
    
# distinguish duplicated keys in different files
    for i in range(len(info)):
        for ii in range(i+1,len(info)):
            if (info[i].split(':')[0] + ':') in info[ii]:
                info[ii] = info[ii].split(':')[0] + '_' + str( "%04d" % ii) + ':' + info[ii].split(':')[1]

# for each file's data, adding filetype symbol
    temp = ''
    for i in ['FILED AS OF DATE','DATE AS OF CHANGE','CONFORMED PERIOD OF REPORT','ACCESSION NUMBER']:
        for ii in range(len(info)):
            if i in info[ii]:
                for iii in range(ii-8,ii+2):
                    if 'CONFORMED SUBMISSION TYPE' in info[iii]:
                        temp = info[iii].split(':')[1]
                info[ii] = temp + '_' + info[ii]
            
# transform to dictionary
    dic = {}
    for i in info:
        try:
            dic.update({i.split(':')[0]:i.split(':')[1]})
        except:
            pass
        
# transform to the dataframe
    df = pd.DataFrame(dic, index = [dic['CENTRAL INDEX KEY']])
    
# put the address together(street, city, state, zip)
    for i, col in enumerate(df.columns):
        if 'ADDRESS' in col:
            df[col] = df[col].str.cat(df.iloc[:,i+1:i+6],sep = ',')
            df.iloc[:, i+1:i+6] = ''
        
    return df

In [37]:
def dropduplicates_comp(infocomp):   
    information = cleaning_comp(infocomp).T.drop_duplicates()   
    return information

In [ ]:
datacomp = pd.DataFrame()
companies = glob.glob('sec_edgar_filings/*')
for company in tqdm(companies):
    path = glob.glob(company + '/*/*.txt')
    infocomp = get_comp(path)
    infocomp = dropduplicates_comp(infocomp)
    datacomp = datacomp.join(infocomp, how = 'outer')
final_comp = datacomp.T

In [ ]:
colnames = final_comp.columns.to_list()
done = []
for i in range(len(colnames)):
    counter = 1
    if colnames[i].split('_')[0] not in done:
        colnames[i] = colnames[i].split('_')[0]
        for ii in range(len(colnames)):
            if colnames[i] in colnames[ii]:
                colnames[ii] = colnames[ii].split('_')[0] + '_' + str(counter)
                counter += 1
        done.append(colnames[i])
final_comp.columns = colnames

In [ ]:
final_comp.fillna('Null',inplace = True)
columns = final_comp.columns.to_list()
iterator = 2
for i in range(final_comp.shape[0]):    
    while (iterator < len(columns)-1):
        if ((columns[iterator].split('_')[0] == columns[iterator-1].split('_')[0]) and (final_comp.iloc[i,iterator-1] == 'Null')):
            final_comp.iloc[i,iterator-1] = final_comp.iloc[i,iterator]
            final_comp.iloc[i,iterator] = 'Null'
        iterator = iterator + final_comp.shape[0]
            
            

In [ ]:
pd.options.display.max_columns = 999
final_comp